0. list all NGOs and all "domains" or "maintainers" or whatever the groups are called...
1. List of available (edu-fin) dataflows would be nice.
2. get 
    - name [done]
    - description [done]
    - dimensions
    - codes
    - maybe else as well 🤷‍♂️

In [1]:
import sdmx
import sdmx1_helpers as hlp

In [2]:
list_of_ngos = sdmx.list_sources() # 0s

# We select an organization

In [3]:
ORGANISATION = 'OECD'

In [4]:
_oecd_df_meta_flat = hlp.get_metadata_for_full_list_of_dataflows(ORGANISATION) # 2 minutes 😱 | 29s | 41s | 11s 😲 | 5.3s | 14s

In [5]:
oecd_df_meta = hlp.restructure_dataflow_meta_dict(_oecd_df_meta_flat) # 0.0 second

In [6]:
#dataflow_constraint_ids = hlp.get_dataflow_constraint_ids("OECD") # 1m13s | 4m7s 😱

In [7]:
dataflow_constraint_ids_single_version = hlp.get_dataflow_constraint_ids_single_version(ORGANISATION) # 1m39s | 48s | 

### here we have selected the dataflow

In [8]:
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
CONSTRAINT_ID = "CR_A_DSD_EAG_UOE_FIN"

In [9]:
dataflow_constraint_ids_single_version[DATAFLOW_ID]

'CR_DSD_EAG_UOE_FIN'

In [10]:
dimension_information = hlp.get_dimension_information(DSD_ID, DATAFLOW_ID) # 2s | 5s | .2s | 2.8s | .1s | 2.9s

In [11]:
constrained_codelists_by_dimensions = hlp.get_constrained_codelists_by_dimensions(ORGANISATION, DSD_ID, DATAFLOW_ID, CONSTRAINT_ID) # 3.4s

In [12]:
dimension_information

{'REF_AREA': {'name': 'Reference area', 'codelist': 'CL_AREA'},
 'MEASURE': {'name': 'Measure', 'codelist': 'CL_MEASURE_FIN'},
 'EDUCATION_LEV': {'name': 'Education level',
  'codelist': 'CL_EDUCATION_LEV_ISCED11'},
 'EXP_SOURCE': {'name': 'Financing source', 'codelist': 'CL_INST_TYPE_EDU'},
 'EXP_DESTINATION': {'name': 'Destination of expenditure',
  'codelist': 'CL_INST_TYPE_EDU'},
 'EXPENDITURE_TYPE': {'name': 'Type of expenditure',
  'codelist': 'CL_EXPENDITURE_TYPE'},
 'PRICE_BASE': {'name': 'Price base', 'codelist': 'CL_PRICES'},
 'UNIT_MEASURE': {'name': 'Unit of measure', 'codelist': 'CL_UNIT_MEASURE'},
 'Q_SHEET': {'name': 'Questionnaire sheet', 'codelist': 'CL_Q_SHEET'},
 'TIME_PERIOD': {'name': 'Time period', 'codelist': None}}

In [ ]:
len(constrained_codelists_by_dimensions['REF_AREA'])

In [ ]:
dim_name = "REF_AREA"
dimension_information[dim_name]['codelist']

In [ ]:
# this should be a list of dataflows that have the same DSD_ID !!!
oecd_df_meta[ORGANISATION]['EDU']['IMEP'][DSD_ID]

# To be sorted out:

In [ ]:
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
# dsd_interesting_attributes = ['codelist', 'concept_scheme', 'dataflow', 'structure']
dsd = sdmx.Client("OECD").datastructure(DSD_ID)
#dataflow_definition = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID]
dataflow_definition = dsd.dataflow[DSD_ID + '@' + DATAFLOW_ID]

# name and description of the dataflow
name = dataflow_definition.name
description = dataflow_definition.description
maintainer = dataflow_definition.maintainer
structure = dataflow_definition.structure
id = dataflow_definition.id
print(name)
print(description)
print(maintainer)
print(structure)
print(id)

In [ ]:
#list_of_ngos
#dataflow_constraint_ids
#oecd_df_meta

# when dataflow is selected:
dimension_information
# we need the codelist + the constraints


# testing for other organisations

# Still experimentation

In [13]:
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
CONSTRAINT_ID = "CR_A_DSD_EAG_UOE_FIN"
#dims = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID].structure.dimensions.components
content_constaints = sdmx.Client("OECD").dataflow(DSD_ID + '@' + DATAFLOW_ID).constraint[CONSTRAINT_ID + '@' + DATAFLOW_ID].data_content_region[0].member.items()

In [12]:
# version is probably not needed, since it is always the latest...
constraint_id = dataflow_constraint_ids_single_version[DATAFLOW_ID]

In [13]:
content_constaints = sdmx.Client("OECD").dataflow(DSD_ID + '@' + DATAFLOW_ID).constraint[constraint_id + '@' + DATAFLOW_ID].data_content_region[0].member.items() # 11s

In [ ]:
content_constaints

In [37]:
def get_constrained_codelists_by_dimensions(organisation, dsd_id, dataflow_id, constraint_id):
    """
    had to dig deep for this one
    scraped from the pits of hell
    """

    content_constaints = sdmx.Client(organisation).dataflow(dsd_id + '@' + dataflow_id).constraint[constraint_id + '@' + dataflow_id]
    
    constrained_codelists_by_dimensions = dict()
    for id, selection in content_constaints.data_content_region[0].member.items():
        # selection.values_for.id == id.id ... whatever a this point ...
        constrained_codelists_by_dimensions[id.id] = selection.values
    
    return constrained_codelists_by_dimensions

In [39]:
constrained_codelists_by_dimensions = get_constrained_codelists_by_dimensions("OECD", DSD_ID, DATAFLOW_ID, CONSTRAINT_ID) # 11s

In [ ]:
constrained_codelists_by_dimensions

In [ ]:
# 'included', 'values', 'values_for']
selection.values_for.id

In [ ]:
selection.values

In [ ]:
# 17 sec
constraint_id = 'CR_A_DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD'
dataflow_id = 'DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD'
df = sdmx.Client("OECD").dataflow(dataflow_id)
cc = df.constraint[constraint_id]
for k, v in cc.data_content_region[0].member.items():
    print(k.id, len(v.values))

print("Example for direct access:")
cc.data_content_region[0].member['REF_AREA'].values

### extracting a full codelist for a given dimension and finding the English name for a given code

In [ ]:
full_codelist_corresponding_to_cl_area = sdmx.Client("OECD").datastructure("DSD_EAG_UOE_FIN").codelist['CL_AREA']

total_number_of_codes_in_cl_area = len(full_codelist_corresponding_to_cl_area)
name_for_key_HUN = full_codelist_corresponding_to_cl_area['HUN'].name['en']

print(f"{total_number_of_codes_in_cl_area=}")
print(f"{name_for_key_HUN=}")


### finding the content constraints for a dataflow, indexing the constraints with a codelist

In [ ]:
# 14 sec
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
CONSTRAINT_ID = "CR_A_DSD_EAG_UOE_FIN"
#dims = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID].structure.dimensions.components
content_constaints = sdmx.Client("OECD").dataflow(DSD_ID + '@' + DATAFLOW_ID).constraint[CONSTRAINT_ID + '@' + DATAFLOW_ID].data_content_region[0].member.items()

In [ ]:
for k, v in content_constaints:
    print(f"{k.id}: {v.values}")

In [ ]:
for dim in content_constaints:
    for dim_code in dim:
        print(f"{dim_code.id}: {dim_code.values}")

In [ ]:
# 17 sec
constraint_id = 'CR_A_DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD' # how to find that??
dataflow_id = 'DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD'
df = sdmx.Client("OECD").dataflow(dataflow_id)
cc = df.constraint[constraint_id]
for k, v in cc.data_content_region[0].member.items():
    print(k.id, len(v.values))

print("Example for direct access:")
cc.data_content_region[0].member['REF_AREA'].values


## List available edu-fin dataflows

In [ ]:
for df_id in dsd.dataflow:
    print(df_id, dsd.dataflow[df_id].name)

# misi kedvéért

In [ ]:
def get_dataflow_constraint_ids(organisation:str)->dict:
    contentconstraints = sdmx.Client(organisation).get("contentconstraint").constraint
    dataflow_constraint_ids = {}
    for constraint_id in contentconstraints:
        try:
            c_id, df_id = constraint_id.split("@")
            version = contentconstraints[constraint_id].version
            if df_id not in dataflow_constraint_ids:
                dataflow_constraint_ids[df_id] = {version: c_id}
            else:
                dataflow_constraint_ids[df_id][version] = c_id
        except:
            pass
        #print(f"{constraint_id:38} : {constraint_obj.name}")
    return dataflow_constraint_ids

In [ ]:
contentconstraints = sdmx.Client("OECD").get("contentconstraint").constraint

In [ ]:
for i, constraint_id in enumerate(contentconstraints):
    if i == 5:
        break

In [ ]:
sdmx.Client(ORGANISATION)

In [ ]:
sdmx.Client("OECD").get("contentconstraint").constraint

In [ ]:
ORGANISATION

In [2]:
# get_dimension_information(DSD_ID, DATAFLOW_ID)

In [ ]:
# 3m2s
conceptscheme = sdmx.Client("OECD").get("conceptscheme") 
for k, v in conceptscheme.codelist.items():
    if 'EDU' in k:
        print(f"{k:38} : {v.id:25}, {v.urn:85}, {v.maintainer}")

In [44]:
# dataflow, constraint
contentconstraint = sdmx.Client("OECD").get("contentconstraint") 


In [ ]:
def get_dataflow_constraint_ids(organisation):
    dataflow_constraint_ids = {}
    for constraint_id in sdmx.Client(organisation).get("contentconstraint").constraint:
        try:
            c_id, df_id = constraint_id.split("@")
            version = contentconstraint.constraint[constraint_id].version
            if df_id not in dataflow_constraint_ids:
                dataflow_constraint_ids[df_id] = {version: c_id}
            else:
                dataflow_constraint_ids[df_id][version] = c_id
        except:
            pass
        #print(f"{constraint_id:38} : {constraint_obj.name}")
    return dataflow_constraint_ids

In [ ]:
dataflow_constraint_ids = {}
for constraint_id in contentconstraint.constraint:
    try:
        c_id, df_id = constraint_id.split("@")
        version = contentconstraint.constraint[constraint_id].version
        if df_id not in dataflow_constraint_ids:
            dataflow_constraint_ids[df_id] = {version: c_id}
        else:
            dataflow_constraint_ids[df_id][version] = c_id
    except:
        pass
    #print(f"{constraint_id:38} : {constraint_obj.name}")
dataflow_constraint_ids

In [ ]:
d = {'1': 'kutyafasza'}
d['3'] = 'macskapöcse'
d

In [ ]:
constraint_id

In [ ]:
contentconstraint.constraint['CR_DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD'].version

In [ ]:
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
CONSTRAINT_ID = "CR_A_DSD_EAG_UOE_FIN"
dataflow_constraint_ids[DATAFLOW_ID]

In [ ]:
contentconstraint.constraint["CR_A_DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD"]

In [ ]:
contentconstraint.constraint["CR_DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD"].version

In [ ]:
for i, item in enumerate(contentconstraint.dataflow.items()):
    if i == 5:
        break

In [ ]:
k, v = item
k

In [ ]:
v.id

In [ ]:
for attr in dir(v):
    if not attr.startswith('_'):
        print(attr, getattr(v, attr))

In [ ]:
contentconstraint.constraint.items()

In [ ]:
contentconstraint.constraint

In [ ]:
constraint_obj.iter_keys

In [ ]:
for attr in dir(constraint_obj):
    if not attr.startswith('_'):
        print(attr, getattr(constraint_obj, attr))

In [ ]:
contentconstraint

In [ ]:
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
CONSTRAINT_ID = "CR_A_DSD_EAG_UOE_FIN"
#dims = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID].structure.dimensions.components
content_constaints = sdmx.Client("OECD").dataflow(DSD_ID + '@' + DATAFLOW_ID).constraint[CONSTRAINT_ID + '@' + DATAFLOW_ID].data_content_region[0].member.items()

In [3]:
# Codelist (664), ConceptScheme (195), DataStructureDefinition (314)
dsd = sdmx.Client("OECD").datastructure

In [4]:
#sdmx.Client("OECD").get("Codelist", "CL_UNIT_MULT")

In [ ]:
root = sdmx.Client("OECD")
for support_name in root.source.supports:
    if root.source.supports[support_name]:
        print(support_name)
print("\nthe only promblem is that i don't know how to call these methods...")

In [ ]:
# objects, dataflow, structure
# time: 24 sec | 15 sec
full_dataflow_detail = root.get("dataflow", detail="full")


In [ ]:
print(len(full_dataflow_detail.structure))
print(len(full_dataflow_detail.dataflow))
full_dataflow_detail.objects

In [ ]:
for item in full_dataflow_detail.structure:
    print(item)

In [20]:
dataflow_meta = dict()
for i, a in enumerate(full_dataflow_detail.dataflow.items()):
    id, name = a
    agency = name.maintainer.__str__()
    try:
        dataflow_name = name.name['en'] # of course: AttributeError: 'DataflowDefinition' object has no attribute 'get' 🙄
    except:
        dataflow_name = None
    dsd_id = name.structure.id
    dsd_urn = name.structure.urn  # what does this urn identify??

    # Handle the case where the agency is not in the dictionary
    if agency not in dataflow_meta:
        dataflow_meta[agency] = dict()
        
    dataflow_meta[agency][dsd_id] = {"dataflow_name": dataflow_name, "dsd_urn": dsd_urn}


In [26]:
def get_dataflow_meta(organization):
    """
    organization = "OECD"
    """
    dataflow_meta = dict()
    
    for i, a in enumerate(sdmx.Client(organization).get("dataflow", detail="full").dataflow.items()):
        id, name = a
        agency = name.maintainer.__str__()
        try:
            dataflow_name = name.name['en'] # of course: AttributeError: 'DataflowDefinition' object has no attribute 'get' 🙄
        except:
            dataflow_name = None
        dsd_id = name.structure.id
        dsd_urn = name.structure.urn  # what does this urn identify??

        # Handle the case where the agency is not in the dictionary
        if agency not in dataflow_meta:
            dataflow_meta[agency] = dict()

        dataflow_meta[agency][dsd_id] = {"dataflow_name": dataflow_name, "dsd_urn": dsd_urn, "id": id}
    return dataflow_meta

In [ ]:
df_meta = get_dataflow_meta("OECD")

In [ ]:
def restructure_dataflow_meta_dict(data):
    """
    This code will reorganize the dictionary to reflect the hierarchical structure based on the keys. 
    The hierarchy dictionary will have nested dictionaries representing the hierarchy of the keys.
    E.g. OECD.WISE.WDP will be represented as {'OECD': {'WISE': {'WDP': {}}}} to reflect the the organization, directorate, and domain levels.
    """
    hierarchy = {}

    for key, value in data.items():
        parts = key.split('.')
        current_level = hierarchy
        
        for part in parts[:-1]:
            if part not in current_level:
                current_level[part] = {}
            current_level = current_level[part]
        
        current_level[parts[-1]] = value

    return hierarchy

In [ ]:
for k in dataflow_meta.keys():
    print(k)

In [ ]:
organization, directorate, domain = agency.split(".")
print(organization, directorate, domain)

In [ ]:
# structure, name, maintainer
name.name['en']           # 'River flooding - Cities and FUAs' | this is the dataflow name
name.structure.urn        # 'urn:sdmx:org.sdmx.infomodel.datastructure.DataStructureDefinition=OECD.CFE.EDS:DSD_FUA_CLIM(1.0)'
name.structure.id         # 'DSD_FUA_CLIM'                     | this is the data structure definition's id
name.structure.version    # '1.0'
name.maintainer.__str__() # 'OECD.CFE.EDS'                     | this is the agency

In [ ]:
name.name['en']

In [ ]:
print(name.structure.maintainer)

In [ ]:
thing = name.structure
for attr in dir(thing):
    if not attr.startswith("_"):
        print(attr, getattr(thing, attr))

In [ ]:
dir(full_dataflow_detail)

In [160]:
agencies = root.get("agencyscheme")
#dir(sdmx.Client("OECD"))

In [ ]:
for agency_group in agencies:
    print(agency_group.id)
    for agency in agency_group:
        print(agency.id, agency.name)

In [ ]:
agencies.organisation_scheme['OECD.EDU:AGENCIES']['IMEP'].id
agencies.organisation_scheme['OECD.EDU:AGENCIES']['IMEP'].name['en']

In [ ]:
agencies.organisation_scheme['OECD.EDU:AGENCIES'].items

In [ ]:
for agency_id in agencies.organisation_scheme:
    print(agency_id, agencies.organisation_scheme[agency_id].items)

In [ ]:
agencies.organisation_scheme

In [ ]:
for attr in dir(agencies):
    if not attr.startswith("_"):
        print(attr, getattr(agencies, attr))

In [ ]:
dir(agencies)

In [ ]:
root.get('')

In [49]:
# interesting_client_methods = [
#   'data', 'dataflow', 'datastructure', 'metadata', 'metadataflow'
# , 'metadatastructure', 'source', 'structure', 'structureset'
# , 'preview_data', 'schema']

### <font color=red>DataFlow related metadata

In [56]:
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
# dsd_interesting_attributes = ['codelist', 'concept_scheme', 'dataflow', 'structure']
dsd = sdmx.Client("OECD").datastructure(DSD_ID)
#dataflow_definition = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID]
dataflow_definition = dsd.dataflow[DSD_ID + '@' + DATAFLOW_ID]

In [ ]:
# fairly useless:
dataflow_definition.get_annotation(title='REF_AREA')

In [ ]:
# name and description of the dataflow
name = dataflow_definition.name
description = dataflow_definition.description
maintainer = dataflow_definition.maintainer
structure = dataflow_definition.structure
id = dataflow_definition.id
print(name)
print(description)
print(maintainer)
print(structure)
print(id)


# Get dimensions and the corresponding codelist names

In [110]:
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
dims = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID].structure.dimensions.components

In [132]:
def get_dimension_information(dsd_id:str, dataflow_id:str)->list[dict[str,str]]:
    """
    Data_Structure_Definition_id and DataFlow_id are required to get the dimension information of the dataflow.
    """


    dims = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID].structure.dimensions.components
    dimensions = list()
    for dim in dims:
        dimension_info = dict()
        dimension_info['dim_code'] = dim.concept_identity.id
        dimension_info['dim_name'] = dim.concept_identity.name['en']
        try:
            dimension_info['corresponding_codelist'] = dim.concept_identity.core_representation.enumerated.id
        except:
            dimension_info['corresponding_codelist'] = None
        dimensions.append(dimension_info)
        
    return dimensions

In [ ]:
get_dimension_information(DSD_ID, DATAFLOW_ID)

In [ ]:

print(f"{'Code':29} -     {'Name':39} -    {'Codelist':20}\n")
for dim in dims:
    dim_code = dim.concept_identity.id
    dim_name = dim.concept_identity.name['en']
    try:
        corresponding_codelist = dim.concept_identity.core_representation.enumerated.id
    except:
        corresponding_codelist = None
    
    print(f"{dim_code=:20} -     {dim_name=:30} -    {corresponding_codelist=}")

### extracting a full codelist for a given dimension and finding the English name for a given code

In [ ]:
full_codelist_corresponding_to_cl_area = sdmx.Client("OECD").datastructure("DSD_EAG_UOE_FIN").codelist['CL_AREA']

total_number_of_codes_in_cl_area = len(full_codelist_corresponding_to_cl_area)
name_for_key_HUN = full_codelist_corresponding_to_cl_area['HUN'].name['en']

print(f"{total_number_of_codes_in_cl_area=}")
print(f"{name_for_key_HUN=}")


### finding the content constraints for a dataflow, indexing the constraints with a codelist

In [117]:
# 14 sec
DSD_ID = "DSD_EAG_UOE_FIN"
DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
CONSTRAINT_ID = "CR_A_DSD_EAG_UOE_FIN"
#dims = sdmx.Client("OECD").datastructure(DSD_ID).dataflow[DSD_ID + '@' + DATAFLOW_ID].structure.dimensions.components
content_constaints = sdmx.Client("OECD").dataflow(DSD_ID + '@' + DATAFLOW_ID).constraint[CONSTRAINT_ID + '@' + DATAFLOW_ID].data_content_region[0].member.items()

In [ ]:
for k, v in content_constaints:
    print(f"{k.id}: {v.values}")

In [ ]:
for dim in content_constaints:
    for dim_code in dim:
        print(f"{dim_code.id}: {dim_code.values}")

In [ ]:
# 17 sec
constraint_id = 'CR_A_DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD' # how to find that??
dataflow_id = 'DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD'
df = sdmx.Client("OECD").dataflow(dataflow_id)
cc = df.constraint[constraint_id]
for k, v in cc.data_content_region[0].member.items():
    print(k.id, len(v.values))

print("Example for direct access:")
cc.data_content_region[0].member['REF_AREA'].values


## List available edu-fin dataflows

In [ ]:
for df_id in dsd.dataflow:
    print(df_id, dsd.dataflow[df_id].name)

## List of NGOs

In [ ]:
list_of_ngos = sdmx.list_sources()
print(f"The first 20 NGOs (from a total of {len(list_of_ngos)} are: ")
print(list_of_ngos[:20])

## 1. Within the organization
### list agencies
We don't yet have the means to filter based on that !!

In [ ]:
# 4 sec
agencyscheme = sdmx.Client("OECD").get("agencyscheme")
for agency_code in agencyscheme.organisation_scheme[organization +':AGENCIES'].items:
    name = agencyscheme.organisation_scheme[organization +':AGENCIES'].items[agency_code].name
    print(agency_code, name)


In [ ]:
# the list of all agencies within oecd:
#for agency_code in agencyscheme.organisation_scheme['OECD:AGENCIES'].items:
def get_agencies(organization:str)->dict:
    agencies = dict()
    agency_items = sdmx.Client(organization).get("agencyscheme").organisation_scheme[organization +':AGENCIES'].items
    for agency_code in agency_items:
        agencies[agency_code] = agency_items[agency_code].name.__str__()
    return agencies

get_agencies("OECD")

## Dimensions
<font color=red>Warning: this is very slow 6m20s

In [111]:
# #DSD_ID = "DSD_EAG_UOE_FIN"
# #DATAFLOW_ID = "DF_UOE_INDIC_FIN_PERSTUD"
# #dataflow_definition_id = DSD_ID + '@' + DATAFLOW_ID
# data = sdmx.Client("OECD").data(dataflow_definition.id)

# for dim in data.structure.dimensions:
#     print(dim.id)

## Codes

In [ ]:
# 1m27s, 1m8s
# what's important from here: urn and id ??? dunno
#conceptscheme = root.get("conceptscheme") 
conceptscheme = sdmx.Client("OECD").get("conceptscheme") 
for k, v in conceptscheme.codelist.items():
    if 'EDU' in k:
        print(f"{k:38} : {v.id:25}, {v.urn:85}, {v.maintainer}")

In [ ]:
# dsd = sdmx.Client("OECD").datastructure("DSD_EAG_UOE_FIN")
for i, item in enumerate(sdmx.Client("OECD").datastructure("DSD_EAG_UOE_FIN").codelist['CL_AREA']):
    if i < 5:
        print(item.id, item.name)

print("It seems that we don't have an urn in here... :/")
# a_codelist = sdmx.Client("OECD").datastructure("DSD_EAG_UOE_FIN").codelist['CL_AREA']
# a_codelist.urn / uri / urn_group


In [ ]:
all_codelist_ids = dsd.codelist.keys()
for code_list_id in all_codelist_ids:
    print(code_list_id, dsd.codelist[code_list_id].name, len(dsd.codelist[code_list_id].items))

### list of constrained code lists (cube reginons or whatever)
we probalby need the urn

In [ ]:
# 17 sec
constraint_id = 'CR_A_DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD'
dataflow_id = 'DSD_EAG_UOE_FIN@DF_UOE_INDIC_FIN_PERSTUD'
df = sdmx.Client("OECD").dataflow(dataflow_id)
cc = df.constraint[constraint_id]
for k, v in cc.data_content_region[0].member.items():
    print(k.id, len(v.values))

print("Example for direct access:")
cc.data_content_region[0].member['REF_AREA'].values


In [ ]:
cc.id